# Multivariate Calculus
## Functions of two variables
Solving for two variables will be very similar to how we solved things in the previous week. Instead of a single variable however we will be using multiple. Suppose we have the following equation:

f(x, y) = 4x^2 - 9xy +6y^3

Let us find partial derivitives for x and y individually. It is incredibly easy with sympy as we saw last week.

In [13]:
import math
from sympy import symbols

x, y = symbols('x, y')
f_x_y = 4 * x**2 - 9 * x * y + 6 * y**3

partial_f_x = f_x_y.diff(x)  # Differentiate our equation with respect to x
print(f"Partial Derivitive of our equation with respect to x: {partial_f_x}")

partial_f_y = f_x_y.diff(y)  # Differentiate our equation with respect to y
print(f"Partial Derivitive of our equation with respect to y: {partial_f_y}")

Partial Derivitive of our equation with respect to x: 8*x - 9*y
Partial Derivitive of our equation with respect to y: -9*x + 18*y**2


We can continue by taking the second derivitive of either. 

In [2]:
partial_y_y = partial_f_y.diff(y) 
partial_y_x = partial_f_y.diff(x)

print(f"Second derivitive fyy(x,y): {partial_y_y}")
print(f"Second derivitive fyx(x,y): {partial_y_x}")

Second derivitive fyy(x,y): 36*y
Second derivitive fyx(x,y): -9


## Finding Extrema
We can also find extrema with multiple variables in the same way we found them last week: use sympy's solve or solveset to get our answer. Suppose we have the equation:

f(x, y) = 6*x^2 + 6 * y^2 + 6 * x * y + 36 * x  - 5

Let us find all critical points for this equation. We begin by getting the partials for x and y. This gives us a system of linear equations. We can use sympy's linsolve function to solve this system of equations for x and y values respectively. 

In [6]:
from sympy import symbols, S, linsolve

x, y = symbols('x, y')

f_x_y = 6*x**2 + 6*y**2 + 6*x*y + 36*x - 5
partial_x = f_x_y.diff(x)
partial_y = f_x_y.diff(y)

results = linsolve([partial_x, partial_y], x, y)

print(f"The x and y value for critical points are {results}")


The x and y value for critical points are FiniteSet((-4, 2))


We now want to see if the critical point we've found is a relative maxima, minima, saddle point, or none of them. We do this by calculating the discriminant D. To help do this, let us create a helper function that implements the discremint formula and use it on our results of (-4, 2)

In [7]:
from sympy import lambdify, Abs

def D(func, x_sym, y_sym, x_crit, y_crit):
    # Calculate the discriminant for a given function
    f_x_x = func.diff(x_sym, x_sym)
    f_y_y = func.diff(y_sym, y_sym)
    f_x_y = func.diff(x_sym, y_sym)
    
    # Create callable functions for each of the derivitives we created
    lambd_x_x = lambdify([x_sym, y_sym], f_x_x)
    lambd_y_y = lambdify([x_sym, y_sym], f_y_y)
    lambd_x_y = lambdify([x_sym, y_sym], f_x_y)

    fxx_ab = lambd_x_x(x_crit, y_crit)
    fyy_ab = lambd_y_y(x_crit, y_crit) 
    fxy_ab = lambd_x_y(x_crit, y_crit)

    d = fxx_ab * fyy_ab - Abs(fxy_ab)**2

    print(f"The discriminant of our critical value is {d}")

    if d < 0:
        print("This is a saddle point")
    elif d > 0:
        if fxx_ab < 0:
            print("This is relative maxima")
        else:
            print("This is a relative minima")
    

# Call our new function and pass in the values
D(f_x_y, x, y, -4, 2)

The discriminant of our critical value is 108
This is a relative minima


Let us try another equation that has multiple points to test. We will use the nonlinsolve this time as the system of equations is nonlinear. You will notice that one of the solutions given includes non-real numbers. We use the is_real parameter to ignore that solution.

f(x, y) = 9xy - x^3 - y^3 - 6

In [8]:
from sympy import symbols, S, nonlinsolve

x, y = symbols('x, y', real=True)

f_x_y = 9*x*y - x**3 - y**3 - 6
partial_x = f_x_y.diff(x)
partial_y = f_x_y.diff(y)

# Get nonlinear solution and remove imaginary numbers
results = nonlinsolve([partial_x, partial_y], [x, y])

print(f"The x and y value for critical points are {results}")

for result in list(results):
    if result[0].is_real and result[1].is_real:  # Ignore any solutions that are not real numbers
        print(f"Analyzing critical point {result}")
        D(f_x_y, x, y, result[0], result[1])  

The x and y value for critical points are FiniteSet((0, 0), (3, 3), ((-3/2 - 3*sqrt(3)*I/2)**2/3, -3/2 - 3*sqrt(3)*I/2), ((-3/2 + 3*sqrt(3)*I/2)**2/3, -3/2 + 3*sqrt(3)*I/2))
Analyzing critical point (0, 0)
The discriminant of our critical value is -81
This is a saddle point
Analyzing critical point (3, 3)
The discriminant of our critical value is 243
This is relative maxima


## Lagrange Multipliers
When solving problems with greater dimensionality, it is helpful to use lagrange multipliers. In python we will create an extra symbol l to represent our new variable lambda. The rest of the process is similar to that above.

Suppose we want to maximize the area of a rectangle x/y with the function f(x, y) = x *y

This is constrained however by the function:

g(x, y) = xy + 20y + 20x + 474000 = 500000

We first set our constraint equal to zero, and then solve for x and y.


In [9]:
from sympy import symbols, nonlinsolve, nsimplify, expand, powsimp

x, y, l = symbols('x, y, l', real=True)

f_xy = x * y
g_xy = x * y + 20 * y + 20 * x + 474000 - 500000  # Constraint is equal to zero when subtracting right side

# Implement lagrange function and solve system
f_xyl = f_xy - l *g_xy
f_xyl_x = f_xyl.diff(x)
f_xyl_y = f_xyl.diff(y)
f_xyl_l = f_xyl.diff(l)

results = nonlinsolve([
    f_xyl_x,
    f_xyl_y,
    f_xyl_l,
], [x, y, l])

# Analyze each of the solutions
for sol in list(results):
    x_sol, y_sol, l_sol = sol

    x_sol = x_sol.evalf()  # Evaluate the value so we're not getting sqrt equation
    y_sol = y_sol.evalf()
    print(f"Max value of x is {x_sol}")
    print(f"Max value of y is {y_sol}")

    print(f"Max area is of ({x_sol}, {y_sol}) is {x_sol * y_sol}")

Max value of x is 142.480768092719
Max value of y is 142.480768092719
Max area is of (142.480768092719, 142.480768092719) is 20300.7692762912
Max value of x is -182.480768092719
Max value of y is -182.480768092719
Max area is of (-182.480768092719, -182.480768092719) is 33299.2307237088


In [21]:

from sympy import symbols
import sympy as sy

x, y = symbols('x, y')
f_x_y = 6 * sy.exp(x**2 - 4*y)

partial_f_x = f_x_y.diff(x)  # Differentiate our equation with respect to x
print(f"Partial Derivitive of our equation with respect to x: {partial_f_x}")

partial_f_y = f_x_y.diff(y)  # Differentiate our equation with respect to y
print(f"Partial Derivitive of our equation with respect to y: {partial_f_y}")

    # Create callable functions for each of the derivitives we created
lambda_xy = lambdify([x, y], f_x_y)
lambd_x = lambdify([x, y], partial_f_x)
lambd_y = lambdify([x, y], partial_f_y)
    
# z = f(x0,y0) + fx(x0,y0)(x-x0) + fy(x0,y0)(y-y0)
print (lambd_y(8,16))


Partial Derivitive of our equation with respect to x: 12*x*exp(x**2 - 4*y)
Partial Derivitive of our equation with respect to y: -24*exp(x**2 - 4*y)
-24.0


You can see that we were given two solutions for (x, y) that maximize the area  with the given constraint. The first solution of (142.5, 142.5) seems to have a lower max area than our second solution. The second solution however uses negative values for x and y which are not possible when calculating the area of a rectangle. 

#6 + 96(x-8) - 24(y-16) 


In [25]:
from sympy import symbols
import sympy as sy

x, y = symbols('x, y')
f_x_y = 2 * sy.sqrt(x*y)

partial_f_x = f_x_y.diff(x)  # Differentiate our equation with respect to x
print(f"Partial Derivitive of our equation with respect to x: {partial_f_x}")

partial_f_y = f_x_y.diff(y)  # Differentiate our equation with respect to y
print(f"Partial Derivitive of our equation with respect to y: {partial_f_y}")

    # Create callable functions for each of the derivitives we created
lambda_xy = lambdify([x, y], f_x_y)
lambd_x = lambdify([x, y], partial_f_x)
lambd_y = lambdify([x, y], partial_f_y)
    
# z = f(x0,y0) + fx(x0,y0)(x-x0) + fy(x0,y0)(y-y0)
print (lambd_x(2,8))

Partial Derivitive of our equation with respect to x: sqrt(x*y)/x
Partial Derivitive of our equation with respect to y: sqrt(x*y)/y
2.0


In [26]:
# 8 + 2(x-2) + 0.5(y-8)

8 + 2*0.1 + 0.5*0.12

8.26

In [32]:
from sympy import symbols, S, nonlinsolve

x, y = symbols('x, y', real=True)

f_x_y = 5*x + 2*y - 8*x**2 - 4*y**2 -9*x*y 
partial_x = f_x_y.diff(x)
partial_y = f_x_y.diff(y)

print(f"Partial Derivitive of our equation with respect to x: {partial_x}")
print(f"Partial Derivitive of our equation with respect to y: {partial_y}")

# Get nonlinear solution and remove imaginary numbers
results = nonlinsolve([partial_x, partial_y], [x, y])

print(f"The x and y value for critical points are {results}")

lambda_xy = lambdify([x, y], f_x_y)

for result in list(results):
    if result[0].is_real and result[1].is_real:  # Ignore any solutions that are not real numbers
        print(f"Analyzing critical point {result}")
        D(f_x_y, x, y, result[0], result[1]) 
        print ( lambda_xy(result[0], result[1]) )

Partial Derivitive of our equation with respect to x: -16*x - 9*y + 5
Partial Derivitive of our equation with respect to y: -9*x - 8*y + 2
The x and y value for critical points are FiniteSet((22/47, -13/47))
Analyzing critical point (22/47, -13/47)
The discriminant of our critical value is 47
This is relative maxima
42/47


In [38]:
from sympy import symbols, nonlinsolve, nsimplify, expand, powsimp

# x = width, y = depth, z = height , base 8, frnt 11, 3 sides 4
x, y, z, l = symbols('x, y, z, l', real=True)

f_xyz = 8*x*y + 15*x*z + 8*y*z 
g_xyz = x * y * z - 200  # Constraint is equal to zero when subtracting right side

# Implement lagrange function and solve system
f_xyzl = f_xyz - l *g_xyz
f_xyzl_x = f_xyzl.diff(x)
f_xyzl_y = f_xyzl.diff(y)
f_xyzl_z = f_xyzl.diff(z)
f_xyzl_l = f_xyzl.diff(l)

results = nonlinsolve([
    f_xyzl_x,
    f_xyzl_y,
    f_xyzl_z,
    f_xyzl_l,
], [x, y, z, l])

# Analyze each of the solutions
for sol in list(results):
    x_sol, y_sol, z_sol, l_sol = sol

    x_sol = x_sol.evalf()  # Evaluate the value so we're not getting sqrt equation
    y_sol = y_sol.evalf()
    z_sol = z_sol.evalf()
    print(f"Max value of x is {x_sol}")
    print(f"Max value of y is {y_sol}")
    print(f"Max value of z is {z_sol}")

    #print(f"Max area is of ({x_sol}, {y_sol}) is {x_sol * y_sol}")
    print(f"Max area is of ({x_sol}, {y_sol}, {z_sol}) is {x_sol * y_sol * z_sol}")
    

Max value of x is 4.74252440598675
Max value of y is 8.89223326122516
Max value of z is 4.74252440598675
Max area is of (4.74252440598675, 8.89223326122516, 4.74252440598675) is 200.000000000000
Max value of x is -2.37126220299338 - 4.10714661365223*I
Max value of y is -4.44611663061258 - 7.70089990059793*I
Max value of z is -2.37126220299338 - 4.10714661365223*I
Max area is of (-2.37126220299338 - 4.10714661365223*I, -4.44611663061258 - 7.70089990059793*I, -2.37126220299338 - 4.10714661365223*I) is (-4.44611663061258 - 7.70089990059793*I)*(-2.37126220299338 - 4.10714661365223*I)**2
Max value of x is -2.37126220299338 + 4.10714661365223*I
Max value of y is -4.44611663061258 + 7.70089990059793*I
Max value of z is -2.37126220299338 + 4.10714661365223*I
Max area is of (-2.37126220299338 + 4.10714661365223*I, -4.44611663061258 + 7.70089990059793*I, -2.37126220299338 + 4.10714661365223*I) is (-4.44611663061258 + 7.70089990059793*I)*(-2.37126220299338 + 4.10714661365223*I)**2


In [45]:
from sympy import symbols, nonlinsolve, nsimplify, expand, powsimp

x, y, l = symbols('x, y, l', real=True)

f_xy = 2*x**2 + x*y + 8*y**2 + 800 
g_xy = x + y - 1700  # Constraint is equal to zero when subtracting right side

lambda_xy = lambdify([x, y], f_xy)

# Implement lagrange function and solve system
f_xyl = f_xy - l *g_xy
f_xyl_x = f_xyl.diff(x)
f_xyl_y = f_xyl.diff(y)
f_xyl_l = f_xyl.diff(l)

results = nonlinsolve([
    f_xyl_x,
    f_xyl_y,
    f_xyl_l,
], [x, y, l])

# Analyze each of the solutions
for sol in list(results):
    x_sol, y_sol, l_sol = sol

    x_sol = x_sol.evalf()  # Evaluate the value so we're not getting sqrt equation
    y_sol = y_sol.evalf()
    print(f"Max value of x is {x_sol}")
    print(f"Max value of y is {y_sol}")

    print(f"Max area is of ({x_sol}, {y_sol}) is {lambda_xy(x_sol, y_sol)}")

print ( lambda_xy(1417, 283) )

Max value of x is 1416.66666666667
Max value of y is 283.333333333333
Max area is of (1416.66666666667, 283.333333333333) is 5058300.00000000
5058301
